In [1]:
import sys; sys.path.insert(0, '..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Add 3-year dementia outcome label

#### For RUN DMC, SCANS and HARMONISATION

In [ ]:
from Utils.data_preparation import adjust_skewness_for_dataframe

## Specify the correct filepath and cohort number
output_df = pd.read_csv('/Users/lirui/Downloads/Cohort_Dementia_Prediction/Cohort_Data/Selected_Data/Data_6.0/original_data/complete_SCANS_121_subjects.csv')
cohort_number = 2

output_df.insert(0, "cohort", pd.DataFrame([cohort_number]*output_df.shape[0]))
output_df['cohort_dementia_final'] = output_df['cohort'].astype(str) + "_" + output_df['dementia_final'].astype(str)

# Select a duration (T) of interest
for T in [3,4]:
    dementia_label = 'dementia_{}yr'.format(str(T))
    output_df[dementia_label] = pd.NA

    output_df['int_T_survival'] = output_df['T_survival'].round()
    for i in range(output_df.shape[0]):
        this_patient_data = output_df.loc[i]
        demented_within_N = (this_patient_data['dementia_final']==1) and (this_patient_data['int_T_survival']<=T)
        censored_after_N = (this_patient_data['dementia_final']==0) and (this_patient_data['int_T_survival']>=T)
        demented_after_N = (this_patient_data['dementia_final']==1) and (this_patient_data['int_T_survival']>T)
        censored_within_N = (this_patient_data['dementia_final']==0) and (this_patient_data['int_T_survival']<T)

        if demented_within_N:
            output_df.loc[i, dementia_label] = 1
        elif (censored_after_N or demented_after_N):
            output_df.loc[i, dementia_label] = 0

    output_df['cohort_'+dementia_label] = output_df['cohort'].astype(str) + "_" + output_df[dementia_label].astype(str)

output_df = adjust_skewness_for_dataframe(output_df)

# Add binarised CMB
mb_bin = []
for num in output_df['num_mb']:
    if pd.isna(num) or num==0:
        mb_bin.append(num)
    elif num>0:
        mb_bin.append(1)
    else:
        print('Error value')
print(np.unique(mb_bin))
output_df['mb_bin'] = mb_bin

# Add GM Fraction
output_df['GMF'] = output_df['GM_vol_ml']/output_df['TBV_ml']
print(output_df.shape)
output_df.head()

In [7]:
output_df.to_csv('/Users/lirui/Downloads/Cohort_Dementia_Prediction/Cohort_Data/Selected_Data/Data_6.0/augmented_data/augmented_complete_SCANS_121_subjects.csv', index=False)